# 12. Writing a Simple Publisher and Subscriber (Python)
http://wiki.ros.org/ROS/Tutorials/WritingPublisherSubscriber%28python%29

## setup environment

In [1]:
import os
import sys 

home_dir = os.getcwd()
home_dir

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level'

In [2]:
# get ROS related environment variables
envs = !printenv | grep ROS 
ROS_ENVs = {}

for env in envs:
    env = env.split('=')
    [k, v] = env
    ROS_ENVs[k] = v

ROS_ENVs['LD_LIBRARY_PATH'] = '/opt/ros/kinetic/lib'
ROS_ENVs['PKG_CONFIG_PATH'] = '/opt/ros/kinetic/lib/pkgconfig'
ROS_ENVs['CMAKE_PREFIX_PATH'] = '/opt/ros/kinetic'  
del ROS_ENVs['PWD']

In [3]:
# define workspace path
ws_dir = 'catkin_ws'
ws_dir_abs = os.path.join(home_dir, ws_dir)
src_dir_abs = os.path.join(ws_dir_abs, 'src') 

# append environment paths
def gen_paths(ws_dir_abs, src_dir_abs):
    paths = {}
    paths['ROS_PACKAGE_PATH'] = src_dir_abs
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['ROSLISP_PACKAGE_DIRECTORIES'] = os.path.join(ws_dir_abs + 'devel/share/common-lisp')
    paths['PKG_CONFIG_PATH'] = os.path.join(ws_dir_abs + 'devel/lib/pkgconfig')
    paths['CMAKE_PREFIX_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel')
    return paths

def append_paths(ROS_ENVs, paths): 
    for path in ROS_ENVs.keys():
        os.environ[path] = ROS_ENVs[path]
        if path in paths.keys():
            os.environ[path] = '{}:{}'.format(paths[path], os.environ[path])

append_paths(ROS_ENVs, 
             gen_paths(ws_dir_abs, src_dir_abs))

In [4]:
# define package name and path
package_name = 'beginner_tutorials'

package_dir_abs = !rospack find {package_name}
package_dir_abs = package_dir_abs[0]
package_dir_abs

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/src/beginner_tutorials'

## Writing the Publisher Node
"Node" is the ROS term for an executable that is connected to the ROS network. Here we'll create the publisher ("talker") node which will continually broadcast a message.

Change directory into the beginner_tutorials package, you created in the earlier tutorial, creating a package:

In [5]:
cd {package_dir_abs}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/src/beginner_tutorials


### The Code
First lets create a 'scripts' folder to store our Python scripts in:

In [6]:
mkdir -p scripts

In [7]:
cd scripts

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/src/beginner_tutorials/scripts


Then download the example script talker.py to your new scripts directory and make it executable:

In [9]:
!wget https://raw.github.com/ros/ros_tutorials/kinetic-devel/rospy_tutorials/001_talker_listener/talker.py

--2018-03-10 21:06:50--  https://raw.github.com/ros/ros_tutorials/kinetic-devel/rospy_tutorials/001_talker_listener/talker.py
Resolving raw.github.com (raw.github.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.github.com (raw.github.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/ros/ros_tutorials/kinetic-devel/rospy_tutorials/001_talker_listener/talker.py [following]
--2018-03-10 21:06:51--  https://raw.githubusercontent.com/ros/ros_tutorials/kinetic-devel/rospy_tutorials/001_talker_listener/talker.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2217 (2.2K) [text/plain]
Saving to: 'talker.py'

talker.py           100%[================

In [11]:
!chmod +x talker.py

In [12]:
cat talker.py

#!/usr/bin/env python
# Software License Agreement (BSD License)
#
# Copyright (c) 2008, Willow Garage, Inc.
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions
# are met:
#
#  * Redistributions of source code must retain the above copyright
#    notice, this list of conditions and the following disclaimer.
#  * Redistributions in binary form must reproduce the above
#    copyright notice, this list of conditions and the following
#    disclaimer in the documentation and/or other materials provided
#    with the distribution.
#  * Neither the name of Willow Garage, Inc. nor the names of its
#    contributors may be used to endorse or promote products derived
#    from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
# "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
# LIMITED TO, T

### The Code Explained
Now, let's break the code down.
```
#!/usr/bin/env python
```

Every Python ROS Node will have this declaration at the top. The first line makes sure your script is executed as a Python script.
```
import rospy
from std_msgs.msg import String
```
You need to import rospy if you are writing a ROS Node. The std_msgs.msg import is so that we can reuse the std_msgs/String message type (a simple string container) for publishing.
```
pub = rospy.Publisher('chatter', String, queue_size=10)
rospy.init_node('talker', anonymous=True)
```    
This section of code defines the talker's interface to the rest of ROS.  
pub = rospy.Publisher("chatter", String, queue_size=10) declares that your node is publishing to the chatter topic using the message type String. String here is actually the class std_msgs.msg.String.  
The queue_size argument is New in ROS hydro and limits the amount of queued messages if any subscriber is not receiving them fast enough. In older ROS distributions just omit the argument.

The next line, rospy.init_node(NAME, ...), is very important as it tells rospy the **name of your node** -- until rospy has this information, it cannot start communicating with the ROS Master. In this case, your node will take on the name talker. NOTE: the name must be a base name, i.e. it cannot contain any slashes "/".

**anonymous = True** ensures that your node has a unique name by adding random numbers to the end of NAME. Refer to [Initialization and Shutdown - Initializing your ROS Node](http://wiki.ros.org/rospy/Overview/Initialization%20and%20Shutdown#Initializing_your_ROS_Node) in the rospy documentation for more information about node initialization options.
```
rate = rospy.Rate(10) # 10hz
```

This line creates a Rate object rate. With the help of its method sleep(), it offers a convenient way for looping at the desired rate. With its argument of 10, we should expect to go through the loop 10 times per second (as long as our processing time does not exceed 1/10th of a second!)
```
 while not rospy.is_shutdown():
    hello_str = "hello world %s" % rospy.get_time()
    rospy.loginfo(hello_str)
    pub.publish(hello_str)
    rate.sleep()
```

This loop is a fairly standard rospy construct: checking the ***rospy.is_shutdown()*** flag and then doing work. You have to check is_shutdown() to check if your program should exit (e.g. if there is a Ctrl-C or otherwise). In this case, the "work" is a call to pub.publish(hello_str) that publishes a string to our chatter topic.  

The loop calls **rate.sleep()**, which sleeps just long enough to maintain the desired rate through the loop.

(You may also run across rospy.sleep() which is similar to time.sleep() except that it works with simulated time as well (see Clock).)

This loop also calls **rospy.loginfo(str)**, which performs triple-duty: the messages get printed to screen, it gets written to the Node's log file, and it gets written to rosout. **rosout** is a handy for debugging: you can pull up messages using **rqt_console** instead of having to find the console window with your Node's output.

**std_msgs.msg.String** is a very simple message type, so you may be wondering what it looks like to publish more complicated types. The general rule of thumb is that constructor args are in the same order as in the .msg file. You can also pass in no arguments and initialize the fields directly, e.g.
```
msg = String()
msg.data = str
```

or you can initialize some of the fields and leave the rest with default values:
```
String(data=str)
```

You may be wondering about the last little bit:
```
try:
    talker()
except rospy.ROSInterruptException:
    pass
```        
In addition to the standard Python __main__ check, this catches a rospy.ROSInterruptException exception, which can be thrown by rospy.sleep() and rospy.Rate.sleep() methods when Ctrl-C is pressed or your Node is otherwise shutdown. The reason this exception is raised is so that you don't accidentally continue executing code after the sleep().

Now we need to write a node to receive the messages.

## Writing the Subscriber Node 
Download the listener.py file into your scripts directory:

In [13]:
cd {package_dir_abs}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/src/beginner_tutorials


In [14]:
mkdir -p scripts

In [15]:
cd scripts

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/src/beginner_tutorials/scripts


In [16]:
!wget https://raw.github.com/ros/ros_tutorials/kinetic-devel/rospy_tutorials/001_talker_listener/listener.py

--2018-03-10 21:32:08--  https://raw.github.com/ros/ros_tutorials/kinetic-devel/rospy_tutorials/001_talker_listener/listener.py
Resolving raw.github.com (raw.github.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.github.com (raw.github.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/ros/ros_tutorials/kinetic-devel/rospy_tutorials/001_talker_listener/listener.py [following]
--2018-03-10 21:32:09--  https://raw.githubusercontent.com/ros/ros_tutorials/kinetic-devel/rospy_tutorials/001_talker_listener/listener.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2406 (2.3K) [text/plain]
Saving to: 'listener.py'

listener.py         100%[========

In [17]:
!chmod +x listener.py

In [18]:
cat listener.py

#!/usr/bin/env python
# Software License Agreement (BSD License)
#
# Copyright (c) 2008, Willow Garage, Inc.
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions
# are met:
#
#  * Redistributions of source code must retain the above copyright
#    notice, this list of conditions and the following disclaimer.
#  * Redistributions in binary form must reproduce the above
#    copyright notice, this list of conditions and the following
#    disclaimer in the documentation and/or other materials provided
#    with the distribution.
#  * Neither the name of Willow Garage, Inc. nor the names of its
#    contributors may be used to endorse or promote products derived
#    from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
# "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
# LIMITED TO, T

### The Code Explained
The code for listener.py is similar to talker.py, except we've introduced a new callback-based mechanism for subscribing to messages.
```
rospy.init_node('listener', anonymous=True)
rospy.Subscriber("chatter", String, callback)

# spin() simply keeps python from exiting until this node is stopped
rospy.spin()
```

This declares that your node subscribes to the chatter topic which is of type ***std_msgs.msgs.String***. **When new messages are received, callback is invoked with the message as the first argument**.

We also changed up the call to **rospy.init_node()** somewhat. We've added the ***anonymous=True*** keyword argument. ROS requires that each node have a unique name. ***If a node with the same name comes up, it bumps the previous one. This is so that malfunctioning nodes can easily be kicked off the network***. The ***anonymous=True*** flag tells rospy to generate a unique name for the node so that you can have multiple listener.py nodes run easily.

The final addition, **rospy.spin()** simply keeps your node from exiting until the node has been shutdown. Unlike roscpp, **rospy.spin() does not affect the subscriber callback functions**, as those have their own threads.

## Building your nodes
We use CMake as our build system and, yes, you have to use it even for Python nodes. This is to make sure that the autogenerated Python code for messages and services is created.

Go to your catkin workspace and run catkin_make:

In [19]:
cd {ws_dir_abs}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws


In [21]:
!catkin_make

Base path: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws
Source space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/src
Build space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/build
Devel space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/devel
Install space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/install
####
#### Running command: "make cmake_check_build_system" in "/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/build"
####
####
#### Running command: "make -j2 -l2" in "/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/build"
####
[  0%] Built target std_msgs_generate_messages_eus
[  0%] Built target _beginner_tutorials_generate_messages_check_deps_AddTwoInts
[  0%] Built target _beginner_tutorials_generate_messages_check_deps_Num
[  0%] Built target std_msgs_generate_messages_l